In [3]:
## This file contains a prototype idea of trying to learn a value function that represents the viable region
## for a LIP model. 
## Author : Avadesh Meduri
## Date : 20/02/2020

import numpy as np
import IPython
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from matplotlib.animation import FuncAnimation
import pickle as p
import os

import copy
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F


In [116]:
## LIPM Environment

class LipmEnv:
    def __init__(self, h):
        self.omega = np.sqrt(9.81/h)
        self.max_leg_length = 0.6
        self.dt = 0.001
        self.h = h
        self.A = np.matrix([[1, self.dt], [(self.omega**2)*self.dt, 1]])
        self.B = np.matrix([0, -(self.omega**2)*self.dt])
        self.t = 0
                                 
    def integrate_lip_dynamics(self, x_t, u_t):
        ## integrates dynamics for one step
        assert np.shape(x_t) == (2,)
        x_t_1 = np.matmul(self.A, np.transpose(x_t)) + np.matmul(self.B.transpose(), [u_t])
        return x_t_1

    def reset_env(self, x0, u0, epi_time):
        ## initialises environment
        self.t = 0
        self.sim_data = np.zeros((4, int(epi_time/self.dt)+1))
        self.sim_data[:,0][0:2] = x0
        self.sim_data[:,0][2] = u0
        self.sim_data[:,0][3] = self.h
    def step_env(self):
        ## integrates the simulation one step
        self.sim_data[:,self.t + 1][0:2] = self.integrate_lip_dynamics(self.sim_data[:,self.t][0:2],\
                                                   self.sim_data[:,self.t][2])
        self.sim_data[:,self.t + 1][2] = self.sim_data[:,self.t][2]
        self.sim_data[:,self.t + 1][3] = self.sim_data[:,self.t][3] 
        self.t += 1
    
    def set_action(self, u):
        self.sim_data[:,self.t][2] = u
        
    def return_sample_data(self):
        return self.sim_data[:,0:self.t]
           
    def show_episode(self, freq, i_no):
        ## Input:
            ## Freq : frame rate (if freq = 5 one in 5 is shown)
            ## i_no : iteration number 
        sim_data = self.sim_data[:,::freq]

        fig = plt.figure()
        ax = plt.axes(xlim=(-5, 5), ylim=(0, sim_data[:,0][3] + 0.2))
        text_str = "iter - " + str(i_no)
        line, = ax.plot([], [], lw=3)
        def init():
            line.set_data([], [])
            return line,
        def animate(i):
            x = sim_data[:,i][0]
            y = sim_data[:,i][3]
            u = sim_data[:,i][2]
            line.set_data([u,x], [0,y])
            return line,
        props = dict(boxstyle='round', facecolor='wheat', alpha=0.5)
        ax.text(0.05, 0.95, text_str, transform=ax.transAxes, fontsize=15,
        verticalalignment='top', bbox=props)
        
        anim = FuncAnimation(fig, animate, init_func=init,
                                       frames=np.shape(sim_data)[1], interval=25, blit=True)

        plt.close(fig)
        plt.close(anim._fig)
        IPython.display.display_html(IPython.core.display.HTML(anim.to_html5_video()))

    def compute_reward(self, step_time):
        ## Computes the reward after step
        r = 0
        step_data = self.sim_data[:,int(self.t - step_time*1000):int(self.t)].copy()
        step_data[0] = np.subtract(step_data[0], step_data[2])
        min_dist = step_data[0].argmin()
        if abs(step_data[0][min_dist]) < 0.02: ## min distance between COM and COP
            r += 10
        if abs(step_data[1][min_dist]) < 0.02: ## Min velocity when min dist is achieved
            r += 10
        
#         if self.sim_data[:,int(self.t - step_time*1000)-1][2] != self.sim_data[:,int(self.t - step_time*1000)][2]:
#             r -= 1 ## penalises if step is taken
#         else:
#             r += 10 ## rewards if no step is taken
            
        return r


In [12]:
### This block samples and store data using epsillon greedy algorithm

def sample_data(no_episodes, epi_t, h, action_set, value_function, show_episode = False):
    # this function samples data
    env = LipmEnv(h)
    sample_data = []
    for e in range(no_episodes):
        print("running iter number - " + str(e))
        x = [0.0, 4*np.random.random() - 2]
        u0 = action_set[np.random.randint(9)]
        #espillon greedy
        if np.random.random() > 0.2:
            x_in = np.tile([u0 - x[0], x[1], 0],(len(action_set),1)) 
            x_in[:,2] = action_set
            a = np.argmax(value_function(torch.tensor((x_in), dtype=torch.float)).cpu().detach().numpy())
        else:
            a = np.random.randint(9)
        step_time = 0.15
        env.reset_env(x, u0, epi_t)

        ## sars_t = s_t, a_t, r_t, s_t+1
        sars_t = np.zeros(6)
        sars_t[0] = u0 - x[0]
        sars_t[1] = x[1]
        sars_t[2] = action_set[a]
        for t in range(0, int(epi_t*1000) - 1):
            if t % int(step_time * 1000) == 0 and t > 0:
                sars_t[3] = env.compute_reward(step_time)
                env.set_action(env.sim_data[:,env.t][0] + action_set[a]) ## setting action
                sars_t[4] = env.sim_data[:,env.t][2] - env.sim_data[:,env.t][0]
                sars_t[5] = env.sim_data[:,env.t][1]
                sample_data.append(sars_t)
                
                sars_t = np.zeros(6)
                sars_t[0] = env.sim_data[:,env.t][2] - env.sim_data[:,env.t][0]
                sars_t[1] = env.sim_data[:,env.t][1]
                # epsillon greedy
                if np.random.random() > 0.2:
                    x_in = [sars_t[0], sars_t[1], 0].copy()
                    x_in = np.tile(x_in,((len(action_set),1)))
                    x_in[:,2] = action_set
                    a = np.argmax(value_function(torch.tensor((x_in), dtype=torch.float)).cpu().detach().numpy())
                else:
                    a = np.random.randint(9)
    
                sars_t[2] = action_set[a]
        
            env.step_env()
    
        if show_episode: 
            env.show_episode(5, e)
            
    ## shifting reward up since the reward
    sample_data = np.asarray(sample_data)
    sample_data[:,3] = np.roll(sample_data[:,3],-1)
    sample_data = sample_data[0:-2]
    return np.asarray(sample_data)

def store_data(data_array, file_name, dir):
    batch_no = str(len(os.listdir(dir)))
    f = open(dir + file_name + "_" + batch_no + ".pkl", 'wb')
    print("dumping data ...")
    p.dump(data_array, f, -1)  
    f.close()    
    print("finished dumping...")

In [107]:
## this block is for the Q function
class ANN(nn.Module):
    
    def __init__(self, input_size, outputs):
        super(ANN, self).__init__()
        self.l1 = nn.Linear(input_size, 128)
        self.l2 = nn.Linear(128, 256)
        self.l3 = nn.Linear(256, 128)
        self.action_value = nn.Linear(128, outputs)
        
    def forward(self, x):
        l1 = F.relu(self.l1(x))
        l2 = F.relu(self.l2(l1))
        l3 = F.relu(self.l3(l2))
        return self.action_value(l3)
        
        

In [109]:
## This block shows how data sampling is done
device = torch.device("cpu")
dq_sampler = ANN(3, 1).to(device) 
## input to the ANN is u - x (u is cop, x is com location), xd, a_set(possible set of actions)
action_set = np.linspace(-0.2, 0.2, 9)
sample = sample_data(1, 1.5, 0.2, action_set, dq_sampler, False)
# sample[0:20]
## The simulation below shows stepping sequences using an epsilon greedy policy

running iter number - 0


In [110]:
########### To be removed ###############################################################

import keras
from keras.models import Sequential
from keras.layers import Dense

sample = sample_data(10, 1.5, 0.2, action_set, dq_sampler, False)
x_train = sample[:,0:3]
y_train = sample[:,3]

model = Sequential()
model.add(Dense(128, input_dim=3, activation = 'relu'))
model.add(Dense(128, activation= 'relu'))
model.add(Dense(1))

optimizer = keras.optimizers.SGD(learning_rate=0.001, momentum=0.8)
model.compile(loss="mse", optimizer=optimizer)
metrics=['loss']
history = model.fit(x_train, y_train, epochs = 50, batch_size = 16)

running iter number - 0
running iter number - 1
running iter number - 2
running iter number - 3
running iter number - 4
running iter number - 5
running iter number - 6
running iter number - 7
running iter number - 8
running iter number - 9
Epoch 1/50
88/88 [==============================] - 0s 430us/step - loss: 40.6655
Epoch 2/50
88/88 [==============================] - 0s 77us/step - loss: 6.5521
Epoch 3/50
88/88 [==============================] - 0s 121us/step - loss: 7.0706
Epoch 4/50
88/88 [==============================] - 0s 100us/step - loss: 7.1477
Epoch 5/50
88/88 [==============================] - 0s 122us/step - loss: 6.9291
Epoch 6/50
88/88 [==============================] - 0s 129us/step - loss: 7.8549
Epoch 7/50
88/88 [==============================] - 0s 168us/step - loss: 6.1531
Epoch 8/50
88/88 [==============================] - 0s 131us/step - loss: 5.8045
Epoch 9/50
88/88 [==============================] - 0s 117us/step - loss: 5.4444
Epoch 10/50
88/88 [============

In [111]:
######################### To be removed #################################################

ann = ANN(3, 1).to(device)
optimizer = optim.SGD(ann.parameters(), lr= 0.005, momentum=0.8)
x_train_tc = torch.from_numpy(x_train).float()
y_train_tc = torch.from_numpy(y_train).float()

for i in range(200):
    prediction = ann(x_train_tc)
    optimizer.zero_grad()
    loss = F.mse_loss(prediction, y_train_tc)
    loss.backward()
    print(loss)
    optimizer.step()


/home/ameduri/.local/lib/python3.6/site-packages/ipykernel_launcher.py:11: UserWarning: Using a target size (torch.Size([88])) that is different to the input size (torch.Size([88, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  # This is added back by InteractiveShellApp.init_path()


tensor(5.7470, grad_fn=<MseLossBackward>)
tensor(5.9019, grad_fn=<MseLossBackward>)
tensor(7.8112, grad_fn=<MseLossBackward>)
tensor(10.1871, grad_fn=<MseLossBackward>)
tensor(5.8703, grad_fn=<MseLossBackward>)
tensor(5.7834, grad_fn=<MseLossBackward>)
tensor(5.5984, grad_fn=<MseLossBackward>)
tensor(5.7729, grad_fn=<MseLossBackward>)
tensor(5.5357, grad_fn=<MseLossBackward>)
tensor(5.6065, grad_fn=<MseLossBackward>)
tensor(5.5505, grad_fn=<MseLossBackward>)
tensor(5.5110, grad_fn=<MseLossBackward>)
tensor(5.5339, grad_fn=<MseLossBackward>)
tensor(5.4768, grad_fn=<MseLossBackward>)
tensor(5.4830, grad_fn=<MseLossBackward>)
tensor(5.4752, grad_fn=<MseLossBackward>)
tensor(5.4467, grad_fn=<MseLossBackward>)
tensor(5.4503, grad_fn=<MseLossBackward>)
tensor(5.4374, grad_fn=<MseLossBackward>)
tensor(5.4239, grad_fn=<MseLossBackward>)
tensor(5.4240, grad_fn=<MseLossBackward>)
tensor(5.4147, grad_fn=<MseLossBackward>)
tensor(5.4068, grad_fn=<MseLossBackward>)
tensor(5.4053, grad_fn=<MseLossBa

In [117]:
#This block contains the deep Q stepper
class DeepQStepper:
    
    def __init__(self, no_inputs, no_outputs, action_set, h, step_time):
        '''
        Input:
            no_inputs : size of the feature vector into the ANN
            no_outputs: Size of the output array from the ANN
            action_set: The list of all possible actions
            h : height of lipm above ground
            step_time: the step time
        '''
        self.device = torch.device("cpu")
        ## input to the ANN is u - x (u is cop, x is com location), xd, a_set(possible set of actions)
        self.dq_stepper = ANN(no_inputs, no_outputs).to(self.device) 
        ## creating a target network to stabilize training
        self.dq_stepper_tar = copy.deepcopy(self.dq_stepper)
        self.action_set = action_set
        self.h = h
        self.step_time = step_time
        self.dq_stepper_optimizer = optim.SGD(self.dq_stepper.parameters(), lr = 1e-3, momentum=0.8)
        
    def sample_data(self, no_episodes, epi_t, h, action_set, q_function, show_episode = False):
        '''
        This method samples data and returns the data in the SARS form [state, action, reward, state_t+1].
        Input:
            no_episodes : number of episodes of sample data
            epi_t : duration of each episode
            h : height of LIPM from the ground
            action_set : the array of possible actions
            q_function : the ANN that predicts value function given current state, action. Q(s,a)
            show_episode : shows a simulation of the episode.
        '''
        # this function samples data
        env = LipmEnv(h)
        sample_data = []
        for e in range(no_episodes):
#             print("running iter number - " + str(e))
            x = [0.0, 4*np.random.random() - 2]
            u0 = action_set[np.random.randint(9)]
            env.reset_env(x, u0, epi_t)
            #espillon greedy
            x[0] = u0 - x[0]
            if np.random.random() > 0.2:
                a = self.compute_max_Q(x, action_set, q_function)[1]
            else:
                a = np.random.randint(9)

            ## sars_t = s_t, a_t, r_t, s_t+1
            sars_t = np.zeros(6)
            sars_t[0] = u0 - x[0]
            sars_t[1] = x[1]
            sars_t[2] = action_set[a]
            for t in range(0, int(epi_t*1000) - 1):
                if t % int(self.step_time * 1000) == 0 and t > 0:
                    sars_t[3] = env.compute_reward(self.step_time)
                    env.set_action(env.sim_data[:,env.t][0] + action_set[a]) ## setting action
                    sars_t[4] = env.sim_data[:,env.t][2] - env.sim_data[:,env.t][0]
                    sars_t[5] = env.sim_data[:,env.t][1]
                    sample_data.append(sars_t)

                    sars_t = np.zeros(6)
                    sars_t[0] = env.sim_data[:,env.t][2] - env.sim_data[:,env.t][0]
                    sars_t[1] = env.sim_data[:,env.t][1]
                    # epsillon greedy
                    if np.random.random() > 0.2:
                        a = self.compute_max_Q(sars_t[0:2], action_set, q_function)[1]
                    else:
                        a = np.random.randint(9)

                    sars_t[2] = action_set[a]

                env.step_env()

            if show_episode: 
                env.show_episode(5, e)
        
        ## shifting reward up since the reward is obtained in the next step
        ## TODO; Check is this matters
        sample_data = np.asarray(sample_data)
        sample_data[:,3] = np.roll(sample_data[:,3],-1)
        sample_data = sample_data[0:-2]
                        
        return np.asarray(sample_data)
        
    def compute_max_Q(self, x, action_set, q_function):
        '''
        This function returns the max Q value for the given state for the set of possible actions.
        It also returns the action that has maximum value
        Input:
            x : state to be evaluated
            action_set : the array of possible actions
            q_function: model to compute Q value
        '''
        x_in = [x[0], x[1], 0]
        x_in = np.tile(x_in, ((len(action_set),1)))
        x_in[:,2] = action_set
        state_values = q_function(torch.FloatTensor(x_in)).cpu().detach().numpy()
        a_opt = np.argmax(state_values)
        q_opt = np.max(state_values)

        return q_opt, a_opt
    
    def sample_mini_batch(self, buffer, batch_size):
        '''
        This function generates a mini batch by randomly selecting elements from the buffer. 
        It tries to keep atleast 10 elements that are from the latest sample to ensure fast convergence
        Input:
            buffer: buffer array
            batch_size : size of the batch
        '''
        mini_batch = np.zeros((batch_size, np.shape(buffer)[1]))
        for i in range(batch_size):
            if i < batch_size - 10:
                mini_batch[i] = buffer[np.random.randint(0, int(0.7*len(buffer)))]
            else:
                mini_batch[i] = buffer[np.random.randint(int(0.7*len(buffer)), int(len(buffer))-1)]
                
        return mini_batch
        
    def optimize(self, no_iter, no_episodes, epi_t, batch_size, gamma, tau):
        '''
        This method trains the deepQstepper model.
        Input:
            no_episodes: number of episodes worth of data to be stored in buffer
            no_iter: number of iteration of training
            epi_t: duration of one episode in seconds
            batch_size: size of the batch
            gamma : discount factor
            tau: the rate at which the target q function is updated
        '''
        for i in range(no_iter):
            if i == 0:
                buffer = self.sample_data(no_episodes, epi_t, self.h, self.action_set, self.dq_stepper)
            else:
                new_sample = self.sample_data(1 , epi_t, self.h, self.action_set, self.dq_stepper)
                buffer = np.concatenate((buffer, new_sample), axis = 0)
                buffer = buffer[len(new_sample):]

            mini_batch = self.sample_mini_batch(buffer, batch_size)
            X_train = torch.from_numpy(mini_batch[:,0:3]).float()
            Y_train = mini_batch[:,3]

            for j in range(len(mini_batch)):
                Y_train[j] += gamma*self.compute_max_Q(mini_batch[j][4:6], self.action_set, self.dq_stepper_tar)[0]
            
            Y_train = torch.from_numpy(np.reshape(Y_train,(len(Y_train), 1))).float()
            
            prediction = self.dq_stepper(X_train)
            self.dq_stepper_optimizer.zero_grad()
            loss = F.mse_loss(prediction, Y_train)
            loss.backward()
            self.dq_stepper_optimizer.step()
            
            for param, target_param in zip(self.dq_stepper.parameters(), self.dq_stepper_tar.parameters()):
                target_param.data.copy_(tau * param.data + (1 - tau) * target_param.data)
            
            
            ## evaluating the performance of the deepq_stepper at the current iteration
            reward = self.show_performance(epi_t)
            
#             print("running iteration number: " + str(i), "loss:" + str(loss))
            print("running iteration number: " + str(i), "reward:" + str(reward))

        return self.dq_stepper
    
    def save(self, file_name):
        '''
        This saves the dq_stepper
        '''
        torch.save(self.dq_stepper.state_dict(), "../../models/" + file_name)
    
    def show_performance(self, epi_t, x0 = None, show_episode = False):
        '''
        This function returs the reward obtained from the episode using the stepper policy
        It also returns a simulation of stepping using the policy
        epi_t : episode time
        x0 : initial state (position, velocity)
        '''
        env = LipmEnv(self.h)
        if x0:
            x = x0
        else:
            x = [0.0, 4*np.random.random() - 2]
        u0 = action_set[np.random.randint(9)]
        env.reset_env(x, u0, epi_t)
        x[0] = u0 - x[0]
        a = self.compute_max_Q(x, self.action_set, self.dq_stepper)[1]
        episode_reward = 0
        for t in range(0, int(epi_t*1000) - 1):
            if t % int(self.step_time * 1000) == 0 and t > 0:
                episode_reward += env.compute_reward(self.step_time) # compute reward
                env.set_action(env.sim_data[:,env.t][0] + action_set[a]) ## setting action
                x = env.sim_data[:,env.t][0:3].copy()
                x[0] = x[2] - x[0]
                x = x[0:2]
                a = self.compute_max_Q(x, self.action_set, self.dq_stepper)[1]
            env.step_env()
        if show_episode:
            env.show_episode(5, 0)        
        return episode_reward

In [118]:
# this block is for training the dq_stepper
action_set = np.linspace(-0.15, 0.15, 9)
dq_stepper = DeepQStepper(3, 1, action_set, 0.2, 0.15)
dq_stepper.optimize(100, 2, 1.5, 32, 0.8, 0.001)

running iteration number: 0 reward:0
running iteration number: 1 reward:10
running iteration number: 2 reward:0
running iteration number: 3 reward:0
running iteration number: 4 reward:0
running iteration number: 5 reward:0
running iteration number: 6 reward:30
running iteration number: 7 reward:0
running iteration number: 8 reward:0
running iteration number: 9 reward:10
running iteration number: 10 reward:0
running iteration number: 11 reward:0
running iteration number: 12 reward:0
running iteration number: 13 reward:0
running iteration number: 14 reward:10
running iteration number: 15 reward:0
running iteration number: 16 reward:0
running iteration number: 17 reward:30
running iteration number: 18 reward:30
running iteration number: 19 reward:0
running iteration number: 20 reward:20
running iteration number: 21 reward:0
running iteration number: 22 reward:10
running iteration number: 23 reward:0
running iteration number: 24 reward:0
running iteration number: 25 reward:0
running iterat

ANN(
  (l1): Linear(in_features=3, out_features=128, bias=True)
  (l2): Linear(in_features=128, out_features=256, bias=True)
  (l3): Linear(in_features=256, out_features=128, bias=True)
  (action_value): Linear(in_features=128, out_features=1, bias=True)
)

In [119]:
# this block is for testing performance
dq_stepper.show_performance(1.0, x0 = [0.0, 0], show_episode=True)
# dq_stepper.save("dqs")

Your browser does not support the video tag.

10

-0.05798403128825269